In [ ]:
from fastai.vision import *
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter("ignore")
from fastai.callbacks import *

In [ ]:
df = pd.read_csv('/home/g/gv53/smai/train.csv', delimiter=',')
path = '/home/g/gv53/smai/ntrain'
test_path = '/home/g/gv53/smai/ntest'

#this code adds .jpg at the end of csv file because fastai requires df file in this format
for i in range(len(df)):
    a = df['image_file'][i] + '.jpg'
    df['image_file'][i] = a

In [ ]:
tfms = get_transforms()
data = ImageDataBunch.from_df(path, df= df, ds_tfms=tfms, size=224, bs=32).normalize()
print("Size of the image is")
print(data.one_batch()[0].size())
print(data.classes)

In [ ]:
learn = cnn_learner(data, models.v, metrics=accuracy)
learn.lr_find()
learn.recorder.plot()
fig = learn.recorder.plot(return_fig=True)
learn.fit_one_cycle(4)

learn.fit_one_cycle(100,slice(1e-3,3e-4), pct_start=0.05)   
learn.save('tom-resnet-grayscale')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

interp.plot_top_losses(9, figsize=(15,11))
plt.savefig('tom.png')

interp.plot_confusion_matrix(figsize=(12,12), dpi=60)
plt.savefig('confusion_matrix.png')
print(accuracy(*learn.TTA()))
interp.most_confused(min_val=2)